# A/B Testing - Final Project

In [10]:
import pandas as pd
import numpy as np
from __future__ import division

In [16]:
# data on control and experiment groups
control = pd.read_csv('../data/control.tsv', sep='\t')
exp = pd.read_csv('../data/experiment.tsv', sep='\t')

# baseline metrics
daily_pageviews = 40000
daily_clicks = 3200
daily_enrollments = 660
ctp = daily_clicks / daily_pageviews
prob_enroll_given_click = daily_enrollments / daily_clicks
prob_payment_given_enroll = .53
prob_payment_given_click = prob_enroll_given_click * prob_payment_given_enroll

## Metric Choice

### Invariant Metrics

**Number of cookies** - the number of unique visitors of the course overview page shouldn't vary between the two groups as we're testing the new feature, since at this point the change we've introduced isn't visible to anyone.

**Number of clicks** - the number of unique cookies that clicked the ***Start free trial*** button should also stay the same, because the click event happens before the free trial screener is triggered.

**Click-through probability** - the number of unique cookies to click the ***Start free trial*** button divided by the number of unique cookies to view the course overview page; this metric shouldn't be affected by the change we're testing: visitors can't see the new feature until they click the ***Start free trial*** button.

### Evaluation Metrics

**Gross conversion** - the number of user IDs to complete checkout and enroll in the free trial divided by the number of unique cookies to click the ***Start free trial*** button. $d_{min} = 0.01$

**Retention** - the number of user IDs to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by the number of user IDs to complete checkout. $d_{min} = 0.01$

**Net conversion** - the number of user IDs to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by the number of unique cookies to click the ***Start free trial*** button. $d_{min} = 0.0075$

where $d_{min}$ is the practical significance level

To introduce the new feature we're testing, we'll be looking for the following experiment outcome: 

1. a significant decrease in gross conversion (meaning we're efficiently filtering out students who aren't ready to dedicate at least 5 hours to their weekly studies);
2. no significant decrease in net conversion (any growth is also okay in our scenario, but it's not required);
3. since retention is effectively a net conversion / gross conversion ratio and we expect the denominator to grow smaller and the numerator to stay the same, this metric should grow accordingly.

### Unused

The **number of user IDs** could potentially be an evaluation metric, since it essentially measures the number of enrollments, which is what we're interested in. However, the number of user IDs is just a raw count and wouldn't work well if we had a different number of cookies in the control and experiment groups (and it's often the case). In fact, gross conversion would be a better estimate of the number of enrollments since it's normalized by the total number of unique cookies in each group, and we've already chosen it as one of the evaluation metrics.

## Measuring Variability

To get the analytical estimate of the standard deviation for each metric, we'll be using the following formula:

$\sigma = \sqrt{\frac{p(1 - p)}{N}}$

where:

$\sigma$ - standard deviation

$p$ - probability

$N$ - sample size

### Gross Conversion

In [89]:
# function for computing the standard deviation analytically
def std(prob, sample_size):
    return np.sqrt(prob * (1 - prob) / sample_size)

In [91]:
# as specified in the experiment description
n_cookies_overview_page = 5000

# number of users who see the Start free trial page
n_cookies_free_trial_page = n_cookies_overview_page * ctp

round(std(prob_enroll_given_click, n_cookies_free_trial_page), 4)

0.0202

In this case, the unit of diversion (cookie) matches the unit of analysis (that is the denominator of the formula for computing gross convesrion), so we expect the analytical estimate to be close to the empirically calculated standard deviation.

### Retention

In [93]:
# number of users who enrolled in the free trial
n_cookies_complete_checkout = n_cookies_overview_page * ctp * prob_enroll_given_click

round(std(prob_payment_given_enroll, n_cookies_complete_checkout), 4)

0.0549

Here, the unit of diversion (cookie) and the unit of analysis (user ID) are different, so the analytically computed standard deviation will highly likely be lower than the empirical value. It might be a good idea to compute it empirically as well to double-check.

### Net Conversion

In [94]:
# using the same variable, n_cookies_free_trial_page, here as the denominators of gross and net conversions are the same
round(std(prob_payment_given_click, n_cookies_free_trial_page), 4)

0.0156

As in the case of the gross conversion, the unit of diversion and the unit of analysis are both a cookie, so we expect the analytical estimate to match the empirical value.

## Sizing

### Number of Samples vs Power

I decided not to use the Bonferroni correction, since it's too conservative for our case (detailed reasoning can be found in the **Summary** section).

To find out the number of samples needed in each case, I used this [online calculator](http://www.evanmiller.org/ab-testing/sample-size.html) (linked in the instructors' notes) with $\alpha = 0.05$ (= 5%) and $\beta = 0.2$ (= 20%) (per quiz instructions), and individual baseline conversion rates and minimum detectable effects ($d_{min}$) given above.

#### Gross Conversion

baseline conversion rate - prob_enroll_given_click = 0.20625 * 100 = 20.625%

$d_{min}$ = 0.01 * 100 = 1%

The resulting sample size is **25835**. To get the total number of pageviews out of it we need to divide it by the click-through probability (which is essentially a clicks/pageviews ratio). However, what we get after this is the total number of pageviews per group, and we have two (control and experiment), so doubling it will give is what we want.

In [37]:
total_pv_gross_conv = 25835 / ctp * 2
total_pv_gross_conv

645875.0

#### Retention

baseline conversion rate - prob_payment_given_enroll = 0.53 * 100 = 53%

$d_{min}$ = 0.01 * 100 = 1%

The resulting sample size is **39115**. To get the total number of pageviews, we first divide it by the enrollments/pageviews ratio and then double it.

In [36]:
total_pv_retention = 39115 / (daily_enrollments / daily_pageviews) * 2
round(total_pv_retention)

4741212.0

#### Net Conversion

baseline conversion rate - prob_payment_given_click = 0.1093125 * 100 = 10.93125%

$d_{min}$ = 0.0075 * 100 = .75%

The resulting sample size is **27413**. By performing the same operation as for gross conversion, we get the total pageviews.

In [38]:
total_pv_gross_conv = 27413 / ctp * 2
total_pv_gross_conv

685325.0

### Duration vs Exposure

If we take a look at the necessary pageviews, we can see the figure for retention is much higher than the other two. Given 40000 pageviews per day, even if we direct 100% of the site traffic to the control and experiment groups, it still takes 4741212 / 40000 = **119** days (almost 4 months!) to run the experiment. Given such a long time span and a high (maximum, in fact) exposure, we might want to revisit our initial choice of evaluation metrics and drop retention, leaving only gross and net conversions. Of the two metrics remaining, the maximum required number of pageviews is **685325** (for net conversion), so we'll stick with this further down the road.

The proposed feature we're testing isn't associated with collecting any sensitive information, and no one can be harmed as a result of this experiment, so it's a low-risk one. It means it's safe to assign the whole daily site traffic to the control and experiment groups (50% of the traffic to each) to get the results quicker. With 40000 daily pageviews, it would take 685325 / 40000 = **18** (rounding up is necessary as we can't collect data for only a part of day) days to run the experiment.

## Experiment Analysis

In [97]:
# function for performing sanity check on invariant metrics
def sanity_check(cont_total, exp_total, prob=.5, zscore=1.96, cnt=True):
    stddev = std(prob, cont_total + exp_total) if cnt else std(cont_total, control.Pageviews.sum())
    me = stddev * zscore
    boundaries = map(lambda x: round(x, 4), [prob - me, prob + me] if cnt else [cont_total - me, cont_total + me])
    observed_value = round(cont_total / (cont_total + exp_total) if cnt else exp_total, 4)
    print 'The observed value is {}'.format(observed_value)
    print '95% confidence interval is {}'.format(boundaries)
    return 'Sanity check passed. The observed value is within the confidence interval.' if\
            boundaries[0] <= observed_value <= boundaries[1] else 'Sanity check failed.\
            The confidence interval doesn\'t contain the observed value.'

### Sanity Checks

In this section, we need to make sure the invariant metrics we chose at the beginning didn't change too much between the control and experiment groups.

#### Number of Cookies

In [98]:
pv_total_cont = control.Pageviews.sum()
pv_total_exp = exp.Pageviews.sum()

sanity_check(pv_total_cont, pv_total_exp)

The observed value is 0.5006
95% confidence interval is [0.4988, 0.5012]


'Sanity check passed. The observed value is within the confidence interval.'

#### Number of Clicks

In [99]:
clicks_total_cont = control.Clicks.sum()
clicks_total_exp = exp.Clicks.sum()

sanity_check(clicks_total_cont, clicks_total_exp)

The observed value is 0.5005
95% confidence interval is [0.4959, 0.5041]


'Sanity check passed. The observed value is within the confidence interval.'

#### Click-Through Probability

Here, I'm building a CI around the click-through probability in the control group and then checking whether the CTP in the experiment group falls within this range, which would mean there was no statistically significant change in the metric.

In [100]:
ctp_cont = clicks_total_cont / pv_total_cont
ctp_exp = clicks_total_exp / pv_total_exp

sanity_check(ctp_cont, ctp_exp, cnt=False)

The observed value is 0.0822
95% confidence interval is [0.0812, 0.083]


'Sanity check passed. The observed value is within the confidence interval.'

All the three invariant metrics have passed the sanity check, so we can be sure the experiment has been conducted properly.

### Effect Size Tests

In this section, we'll need to build a couple of confidence intervals around the differences between the experiment group and the control group to find out if the changes are statistically and/or practically significant.

In [123]:
# function for building confidence intervals and testing effect size
def build_ci(n_cont, x_cont, n_exp, x_exp, zscore=1.96):
    p_pooled = (x_cont + x_exp) / (n_cont + n_exp)
    se_pooled = np.sqrt(p_pooled * (1 - p_pooled) * (1 / n_cont + 1 / n_exp))
    d_hat = x_exp / n_exp - x_cont / n_cont
    print 'The observed difference is {}'.format(round(d_hat, 4))
    boundaries = map(lambda x: round(x, 4), [d_hat - zscore * se_pooled, d_hat + zscore * se_pooled])
    return '95% confidence interval is {}'.format(boundaries)

#### Gross Conversion

In [124]:
n_clicks_cont_trunc = control[~pd.isnull(control.Enrollments)].Clicks.sum()
n_clicks_exp_trunc = exp[~pd.isnull(exp.Enrollments)].Clicks.sum()

n_enroll_cont = control.Enrollments.sum()
n_enroll_exp = exp.Enrollments.sum()

build_ci(n_clicks_cont_trunc, n_enroll_cont, n_clicks_exp_trunc, n_enroll_exp)

The observed difference is -0.0206


'95% confidence interval is [-0.0291, -0.012]'

The confidence interval doesn't include 0, which means the change is statistically significant. In addition, it doesn't contain the practical significance boundary ($d_{min} = -0.01$) - it signals practical significance as well!

#### Net Conversion

In [125]:
n_payments_cont = control.Payments.sum()
n_payments_exp = exp.Payments.sum()

build_ci(n_clicks_cont_trunc, n_payments_cont, n_clicks_exp_trunc, n_payments_exp)

The observed difference is -0.0049


'95% confidence interval is [-0.0116, 0.0019]'

This change is neither statistically nor practically significant: the CI includes 0 and a negative practical significance boundary ($d_{min} = -0.0075$).

### Sign Tests

They'll be performed with this [online calculator](http://graphpad.com/quickcalcs/binomial1.cfm).

#### Gross Conversion

In [132]:
# merging the control and experiment groups into a single dataframe for convenience
merged = pd.merge(control, exp, on='Date').dropna()

merged['Enrollment_ratio_x'] = merged.Enrollments_x / merged.Clicks_x
merged['Enrollment_ratio_y'] = merged.Enrollments_y / merged.Clicks_y
merged['Payment_ratio_x'] = merged.Payments_x / merged.Clicks_x
merged['Payment_ratio_y'] = merged.Payments_y / merged.Clicks_x

n_days = merged.shape[0]
n_impr_enroll = merged[merged.Enrollment_ratio_y > merged.Enrollment_ratio_x].shape[0]

In [135]:
merged[merged.Enrollment_ratio_y > merged.Enrollment_ratio_x][['Enrollment_ratio_x', 'Enrollment_ratio_y']]

,Enrollment_ratio_x,Enrollment_ratio_y
17,0.209239,0.220109
18,0.265223,0.276479
19,0.227520,0.284341
20,0.246459,0.252078


We can see better results in the experiment group in 4 cases out of 23. With the probability 0.5, the online calculator linked above gives a p-value of **0.0026**. It's smaller than $\alpha = 0.05$, which corroborates the findings of the effect size test (that is the change is statistically significant).

#### Net Conversion

In [136]:
n_impr_pay = merged[merged.Payment_ratio_y > merged.Payment_ratio_x].shape[0]
merged[merged.Payment_ratio_y > merged.Payment_ratio_x][['Payment_ratio_x', 'Payment_ratio_y']]

,Payment_ratio_x,Payment_ratio_y
1,0.089859,0.116816
4,0.076464,0.112306
8,0.086831,0.111433
9,0.112660,0.113821
12,0.084211,0.102256
17,0.123641,0.163043
18,0.116373,0.129905
20,0.143059,0.174221
21,0.136564,0.146843
22,0.096681,0.148629


Here, the experiment group boasts higher values in 10 cases out of 23. The calculator gives a p-value of **0.6776**, once again confirming the results we obtained from the effect size tests (net conversion doesn't show any statistically significant change).

### Summary

As I mentioned above, I decided not to use the Bonferroni correction. It's used in performing multiple comparisons to control for type I errors, which we should be concerned with when a statistically significant difference in any metric can trigger the launch of the new feature (in other words, a single metric governs the overall decision). However, in our case, the two evaluation metrics must both display statistically significant differences for us to proceed with the launch. To paraphrase, by controlling for the false positive rate, we're actually increasing the false negative rate, which in our situation decreases the probability of launching the new feature when we have perfectly valid reasons to do so. Therefore, using the Bonferroni correction is inappropriate here.

The results of the effect size tests and sign tests seem to agree and show that only gross conversion has changed in a statistically significant way.

## Recommendation

The results show a significant decrease of 2% in gross conversion and an insignificant decrease of 0.5% in net conversion. If we take a closer look at the observed difference in net conversion between the experiment and control groups, the fact that it's negative is somewhat of a red flag. Moreover, a 95% confidence interval ([-0.0116, 0.0019]) built for this difference lies mainly below zero and includes the negative practical significance boundary ($d_{min} = -0.0075$). What it means is that if Udacity introduces the new feature based on the result of this experiment, it's actually running a risk of reducing the number of paying students and therefore incurring financial losses. Thus, my recommendation would be not to proceed with the launch.

## Follow-Up Experiment

The follow-up experiment I'm suggesting can be easily built upon the existing infrastructure. The experiment we've been analyzing attempts to filter out students who are not ready to commit 5 hours or more to studying each week, thus reducing the number of potential cancellations. However, the lack of time is not the only thing that leads to dropping out of a course. A more important, if not the most important, factor is whether a student has the necessary background knowledge and prerequisite skills. If these expectations aren't clearly communicated before the enrollment, a student may find themselves struggling with the content even if they put much more than 5 hours weekly, which leads to an ever growing frustration and subsequent cancellation. To avoid this situation, I suggest testing the following feature: when a student clicks the ***Start free trial*** button, they are offered a short test (depending on a course, it might be a multiple choice quiz or a programming assignment or something along those lines that doesn't take too much time to complete). If the student passes the quiz, they're enrolled in the free trial since they've proven they have what it takes to be successful in the course. If, however, the student's having troubles with the pre-enrollment test, a pop-up window shows up offering a list of resources (or probably even a list of other free Udacity courses) the student might find useful in preparing for the course; a recommendation is given to the student to study the offered resources and get a little practice and then come back to re-enroll in the paid course, or to proceed with the course, but opt for the free version instead. The hypothesis here is that this feature will be instrumental in helping filter out the students who might get frustrated mid-trial because of a high course difficulty and cancel their enrollment. I believe it can get even more effective when combined with the free trial screener that we tested in the main experiment.

A detailed setup is given below:

**Hypothesis** - the suggested pre-enrollment test will successfully discourage from enrollment those students who might not have the prerequisite skills and knowledge and who are likely to cancel because the course proves too difficult for them.

**Invariant metrics** - as I mentioned, the follow-up experiment can be built atop the already established infrastructure, so it should be possible to use the same metrics we used in the main experiment: number of clicks, number of cookies, and click-through probability - these are all captured before a user sees the new feature, so they aren't expected to be influenced by it.

**Evaluation metrics** - to introduce the new feature, we'll be looking for a significant decrease in enrollment (gross conversion), which would mean we were able to filter out students who are potentially not ready to take a paid course, coupled with a significant increase in the number of paying students (net conversion), which would justify the introduction of the new feature financially. Unless Udacity is willing and ready to run the experiment for a long time, we won't be measuring retention, as we saw in the main experiment that it takes too many pageviews (over 4.7 million), and consequently a lot of time, to detect a significant change in the metric.

**Unit of diversion** - as in the main experiment, we can use a cookie initially (as the home page visitors aren't necessarily logged in), but once a user enrolls in the free trial, we'll need to switch to tracking them by their user ID.